# Supervised Learning - Naive Bayes

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style="font-size:16px">In this Notebook, We go over some of the Naive Bayes basic definitions and formulas. The workflow of the notbook is explained. Following preprocessing steps and some evaluation metrics. Two Naive Bayes Classifiers were used. The MultinomialNB classifier had the better performance.   </p>

### Achieved Accuracy: 98.8% 


## Formula:
$$P(A|B) = \frac{P(B|A) P(A)}{P(B)}$$

## Example: Spam filters
## $P(Spam | email) = P(Spam | \vec{w}) = \frac{P(spam) \cdot P(\vec{w} | Spam)}{P(spam)\:\cdot \: P(\vec{w}|Spam)\:\cdot \: P(not \:Spam)\: \cdot \: P(\vec{w} | not \:spam)}$

### Terminolgy
Priors --> P(spam)  
Likelihood --> P(spam | $\vec{w}$) and other | terms 
Evidence --> The lower term of the formula $P(spam)\:\cdot \: P(\vec{w}|Spam)\:\cdot \: P(not \:Spam)\: \cdot \: P(\vec{w} | not \:spam)$

## Preprocessing (aka Featurising)
<ol style="font-size:16px">
    <li>Tokenization: splitting the words </li>
    <li>Stop words removal</li>
    <li>Remove non-alphabetical charachters.</li>
    <li>Stemming: keeping the root of the word but stripping things like ing, ed etc. More for large data</li>
    <li>Lemmatization: Alternative to stemming by assinging to the same root. more taxing of resources than stemming</li>
    <li>Lowercasing: could be bad in cases where the name turns into a verb like "Mark" and "mark"</li>

</ol>

## There are 2 phases in the Classifer workflow:

<ol style="font-size:16px">
    <li>Learning phase: splitting the data into training and testing data</li>
    <li>Evaluation phase: testing the classifier performance using key metrics.
        Given that TP/FP is True/False Positive, TN/FN True/False Negative:
        <ol> 
            <li>Accuracy: ratio of correctly predicted observation to the total observations
            $$Accuracy = \frac{TP+TN}{TP+FP+TN+FN}$$</li>
            <li>Precision: is the ratio of correctly predicted positive observations to the total predicted positive observations:
            $$Precision = \frac{TP}{TP + FP}$$</li>
            <li>Recall or Sensitivity: the ratio of correctly predicted positive observations to the all observations in actual class 
            $$Recall = \frac{TP}{TP+FN}$$</li>
        </ol>
    </li>
</ol>

## Building Naive Bayes Classifer

In [ ]:
# library imports
import numpy as np 
import pandas as pd 
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt



## Exploratory Data Analysis

In [ ]:
# reading data
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv")
df = df[["v1", "v2"]]

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# checking unique values on v1
print("Number of Unique Values: \n", df['v1'].value_counts(),'\n\n')

# value percentage
print("Value Percentage: \n",df['v1'].value_counts() * 100 / len(df['v1']))

## Preprocessing

<ol style="font-size:16px">
    <li>Tokenization: splitting the words </li>
    <li>Stop words removal</li>
    <li>Remove non-alphabetical charachters.</li>
    <li>Stemming: keeping the root of the word but stripping things like ing, ed etc. More for large data</li>
    <li>Lemmatization: Alternative to stemming by assinging to the same root. more taxing of resources than stemming</li>
    <li>Lowercasing: could be bad in cases where the name turns into a verb like "Mark" and "mark"</li>

</ol>

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
stop_words = stopwords.words("english")
print(len(stop_words))
print(stop_words[:10])

In [ ]:
# # 1. Tokenization
# df['tokens'] = df.v2.str.split(" ")
# df.head()

In [ ]:
def remove_stopwords(text):
    """Function to replace stopords with an empty space and removes double spaces
    removing all chars other then alphabet and stemming words
    
    returns: corpus of stemmed words"""

    text = re.sub('^a-zA-Z',' ',text)
    text = text.split()
    text = [ps.stem(word) for word in text if word not in stop_words]
    text = ' '.join(text).replace('  ', ' ')
    return text


In [ ]:
df["removed_stopwords"]= df.v2.apply(remove_stopwords)

In [ ]:
df

In [ ]:
# vecotrizing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500)


In [ ]:
X = cv.fit_transform(df.removed_stopwords).toarray()
print(X.shape)

In [ ]:
# target
y = pd.get_dummies(df.v1, drop_first=True)
print(y.shape)

In [ ]:
y = np.array(y)
print(y.shape)

In [ ]:
print(y)

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score, precision_recall_curve, plot_precision_recall_curve

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, shuffle=False)

In [ ]:
X_train.shape

In [ ]:
gauss_clf = GaussianNB()
multi_clf = MultinomialNB()

In [ ]:
gauss_clf.fit(X_train, y_train)
multi_clf.fit(X_train, y_train)

In [ ]:
gauss_pred = gauss_clf.predict(X_test)
multi_pred = multi_clf.predict(X_test)

## Evaluation 

In [ ]:
score_gauss = accuracy_score(gauss_pred, y_test)
score_multi = accuracy_score(multi_pred, y_test)
print(f"GaussNB Accuracy Score: {score_gauss*100:.2f}%")
print(f"MultinomialNB Accuracy Score: {score_multi*100:.2f}%")

In [ ]:
average_precision_gauss = average_precision_score(y_test, gauss_pred)
average_precision_multi = average_precision_score(y_test, multi_pred)

print(f"Avg Precision for GaussianNB: {average_precision_gauss:.2f}")
print(f"Avg Precision for MultinomialNB: {average_precision_multi:.2f}")

In [ ]:
# plotting the MultinomialNB Precision
fig, ax = plt.subplots(figsize=(12,8))
curve = plot_precision_recall_curve(multi_clf, X_test, y_test, ax=ax)
curve.ax_.set_title("MultinomialNB Precision/Recall Curve");
